In [1]:
import numpy as np
import pandas as pd
import os
import glob
from datetime import datetime

In [3]:
path ="/Users/philippheitmann/Desktop/docker_test/data/csv_data"

### Compare result from Jonas Pandas Script with our spark script

#### Recreate Jonas Pandas Skript

In [27]:
# get path of all .csv files (history from API crawling)
all_files = glob.glob(os.path.join(path, "*.csv"))
df = pd.concat((pd.read_csv(f, index_col=0, header=0) for f in all_files))

In [28]:
df = df[(df['time'] >= 1652951631) & (df["time"] <= 1652953679)][[
    "callsign", "geo_altitude", "country_cc", "avg_no_seats", "vertical_rate",
    "time"
]]

In [29]:
df_starting = df[(df.geo_altitude <= 3000)
        & (df.vertical_rate <= -4)]

df_landing = df[(df.geo_altitude <= 3000)
        & (df.vertical_rate >= 4)]



In [30]:
df_starting = df_starting.sort_values(by=['geo_altitude'],
               ascending=True).groupby('callsign', as_index=False).first()

df_landing = df_landing.sort_values(
    by=['geo_altitude'], ascending=True).groupby('callsign',
                                                 as_index=False).first()


In [31]:
df_starting = df_starting.groupby('country_cc').agg({'avg_no_seats': 'sum'})
df_landing = df_landing.groupby('country_cc').agg({'avg_no_seats': 'sum'})

In [32]:
sum_pass = df_landing.avg_no_seats.sub(df_starting['avg_no_seats'],
                                       fill_value=0)


#### Load our results from spark 

In [36]:
path_s = "/Users/philippheitmann/Desktop/TravelDashboard/df_starting.csv"
start_spark = pd.read_csv(path_s, index_col=0)

In [38]:
path_l = "/Users/philippheitmann/Desktop/TravelDashboard/df_landing.csv"
land_spark = pd.read_csv(path_l, index_col=0)

In [42]:
land_spark.columns = ["country_cc", "avg_no_seats"]
start_spark.columns = ["country_cc", "avg_no_seats"]

,country_cc,sum(avg_no_seats)
0,DZ,653.0
1,LT,381.0
2,FI,870.0
3,RO,570.4
4,NL,6278.2
...,...,...
58,AR,532.0
59,PR,1369.8
60,LU,430.0
61,VN,1400.5


In [52]:
land_spark = land_spark.set_index("country_cc", drop=True)
start_spark = start_spark.set_index("country_cc", drop=True)

In [53]:
land_spark.avg_no_seats.sub(start_spark['avg_no_seats'], fill_value = 0)

country_cc
AE    2825.3
AL    -385.1
AR     394.0
AT   -1159.8
AU    -476.7
       ...  
TR   -4044.4
TW    -500.0
US   -6345.2
VN   -1044.4
ZA     -64.4
Name: avg_no_seats, Length: 72, dtype: float64

In [54]:
sum_pass

country_cc
AE    2825.3
AL    -385.1
AR     394.0
AT   -1159.8
AU    -476.7
       ...  
TR   -4044.4
TW    -500.0
US   -6345.2
VN   -1044.4
ZA     -64.4
Name: avg_no_seats, Length: 72, dtype: float64

Seems to work :) 